In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
from utils.classifications_utils import *
from utils.data_processing_utils import *
from utils.data_visualization_utils import *
from utils.metrics_utils import *
from utils.grid_search_utils import *
from sklearn.model_selection import train_test_split
import warnings
%matplotlib inline
plt.style.use('ggplot')
warnings.filterwarnings("ignore")


In [2]:
#specify Project and drone data
project = "Project 2"
drone_in_use = "drone1"

In [3]:
level_0_wind = load_data(2.5, 6, project, drone_in_use, direction="")
level_0_wind = separate_data_based_on_apparatus(level_0_wind)

level_1_wind = load_data(2.5, 6, project, drone_in_use, direction="away")
level_1_wind = separate_data_based_on_apparatus(level_1_wind)

level_2_wind = load_data(2.5, 6, project, drone_in_use, direction="left")
level_2_wind = separate_data_based_on_apparatus(level_2_wind)

level_3_wind = load_data(2.5, 6, project, drone_in_use, direction="right")
level_3_wind = separate_data_based_on_apparatus(level_3_wind)

In [4]:
acc_no_wind, gyro_no_wind, mag, stab = level_0_wind.values()
label_0 = [0 for x in range(acc_no_wind.shape[0])]

acc_level_1_wind, gyro_level_1_wind, mag, stab = level_1_wind.values()
label_1 = [1 for x in range(acc_level_1_wind.shape[0])]

acc_level_2_wind, gyro_level_2_wind, mag, stab = level_2_wind.values()
label_2 = [2 for x in range(acc_level_2_wind.shape[0])]

acc_level_3_wind, gyro_level_3_wind, mag, stab = level_3_wind.values()
label_3 = [3 for x in range(acc_level_3_wind.shape[0])]

In [5]:
X_train_gyro_0, X_test_gyro_0, y_train_gyro_0, y_test_gyro_0 = \
train_test_split(gyro_no_wind, label_0, test_size=0.2, shuffle=False)

In [6]:
X_train_gyro_1, X_test_gyro_1, y_train_gyro_1, y_test_gyro_1 = \
train_test_split(gyro_level_1_wind, label_1, test_size=0.2, shuffle=False)

In [7]:
X_train_gyro_2, X_test_gyro_2, y_train_gyro_2, y_test_gyro_2 = \
train_test_split(gyro_level_2_wind, label_2, test_size=0.2, shuffle=False)

In [8]:
X_train_gyro_3, X_test_gyro_3, y_train_gyro_3, y_test_gyro_3 = \
train_test_split(gyro_level_3_wind, label_3, test_size=0.2, shuffle=False)

In [9]:
X_train_gyro = X_train_gyro_0.append(X_train_gyro_1)
X_train_gyro = X_train_gyro.append(X_train_gyro_2)
X_train_gyro = X_train_gyro.append(X_train_gyro_3)
y_train_gyro = np.hstack((y_train_gyro_0, y_train_gyro_1, y_train_gyro_2, y_train_gyro_3))
train_data = X_train_gyro
sLength = len(train_data)
train_data['labels'] = y_train_gyro

In [10]:
X_test_gyro = X_test_gyro_0.append(X_test_gyro_1)
X_test_gyro = X_test_gyro.append(X_test_gyro_2)
X_test_gyro = X_test_gyro.append(X_test_gyro_3)
test_labels = np.hstack((y_test_gyro_0, y_test_gyro_1, y_test_gyro_2, y_test_gyro_3))
test_data = X_test_gyro
sLength = len(test_data)
test_data['labels'] = test_labels

In [11]:
def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += (size / 2)

def segment_signal(data,window_size = 100):
    segments = np.empty((0,window_size,3))
    labels = np.empty((0))
    for (start, end) in windows(data["labels"], window_size):
        x = data["gyro.x"][start:end]
        y = data["gyro.y"][start:end]
        z = data["gyro.z"][start:end]
        if(len(data["labels"][start:end]) == window_size):
            segments = np.vstack([segments,np.dstack([x,y,z])])
            labels = np.append(labels,stats.mode(data["labels"][start:end])[0][0])
    return segments, labels

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def depthwise_conv2d(x, W):
    return tf.nn.depthwise_conv2d(x,W, [1, 1, 1, 1], padding='VALID')

def apply_depthwise_conv(x,kernel_size,num_channels,depth):
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    return tf.nn.relu(tf.add(depthwise_conv2d(x, weights),biases))
    
def apply_max_pool(x,kernel_size,stride_size):
    return tf.nn.max_pool(x, ksize=[1, 1, kernel_size, 1], 
                          strides=[1, 1, stride_size, 1], padding='VALID')

In [12]:
segments, labels = segment_signal(train_data)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
reshaped_segments = segments.reshape(len(segments), 1,100, 3)
train_x = reshaped_segments
train_y = labels

In [13]:
segments, labels = segment_signal(test_data)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
reshaped_segments = segments.reshape(len(segments), 1,100, 3)
test_x = reshaped_segments
test_y = labels

In [14]:
input_height = 1
input_width = 100
num_labels = 4
num_channels = 3

batch_size = 10
kernel_size = 60
depth = 60
num_hidden = 1000

learning_rate = 0.001
training_epochs = 10

total_batches = train_x.shape[0] // batch_size
display(total_batches)

230

In [15]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])

c = apply_depthwise_conv(X,kernel_size,num_channels,depth)
p = apply_max_pool(c,20,2)
c = apply_depthwise_conv(p,6,depth*num_channels,depth//10)

shape = c.get_shape().as_list()
c_flat = tf.reshape(c, [-1, shape[1] * shape[2] * shape[3]])

f_weights_l1 = weight_variable([shape[1] * shape[2] * depth * num_channels * (depth//10), num_hidden])
f_biases_l1 = bias_variable([num_hidden])
f = tf.nn.tanh(tf.add(tf.matmul(c_flat, f_weights_l1),f_biases_l1))

out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])
y_ = tf.nn.softmax(tf.matmul(f, out_weights) + out_biases)

In [16]:
loss = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
cost_history = np.empty(shape=[1],dtype=float)

with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
            cost_history = np.append(cost_history,c)
        print("Epoch: " + str(epoch) + " Training Loss: " + str(c) + " Training Accuracy: " +
              str(session.run(accuracy, feed_dict={X: train_x, Y: train_y})))
    print("Testing Accuracy:" + str(session.run(accuracy, feed_dict={X: test_x, Y: test_y})))

Epoch: 0 Training Loss: 3.1471376e-05 Training Accuracy: 0.24967434
Epoch: 1 Training Loss: 4.1842683e-05 Training Accuracy: 0.24967434
Epoch: 2 Training Loss: 1.740459e-05 Training Accuracy: 0.24967434
Epoch: 3 Training Loss: 6.544661e-05 Training Accuracy: 0.24967434
Epoch: 4 Training Loss: 0.00047955368 Training Accuracy: 0.24967434
Epoch: 5 Training Loss: 0.00021136228 Training Accuracy: 0.24967434
Epoch: 6 Training Loss: 0.00017333249 Training Accuracy: 0.24967434
Epoch: 7 Training Loss: 0.00012791262 Training Accuracy: 0.24967434
Epoch: 8 Training Loss: 0.00026870973 Training Accuracy: 0.24967434
Epoch: 9 Training Loss: 0.0004916395 Training Accuracy: 0.24967434
Testing Accuracy:0.24869566
